In [ ]:
# anger initial size : 2466
# anger oversample size : 3880.619138389918
# anger final size      : 6346.619138389919

# contempt initial size : 165
# contempt oversample size : 5019.270584122538
# contempt final size      : 5184.270584122538

# disgust initial size : 191
# disgust oversample size : 5006.404466091661
# disgust final size      : 5197.404466091661

# fear initial size : 652
# fear oversample size : 4778.27829639034
# fear final size      : 5430.27829639034

# happiness initial size : 7528
# happiness oversample size : 1375.6849279168543
# happiness final size      : 8903.684927916855

# neutral initial size : 10308
# neutral oversample size : 0.0
# neutral final size      : 10308.0

# sadness initial size : 3514
# sadness oversample size : 3362.015611606874
# sadness final size      : 6876.015611606874

# surprise initial size : 3562
# surprise oversample size : 3338.2627783191
# surprise final size      : 6900.2627783191

In [34]:
# AUGMENTATION

from keras.preprocessing.image import ImageDataGenerator
from skimage import io
import os

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# The base directory where the train data is stored
base_train_dir = '../dataset/fer2013plus/train'

# The base directory where the augmented images will be saved
base_augmented_dir = '../dataset/augmented'

# List of emotions
# emotions = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

emotions = ['surprise']

# Iterate over each emotion
for emotion in emotions:
    # Path to the specific emotion directory
    emotion_dir = os.path.join(base_train_dir, emotion)
    
    # Ensure the augmented directory for the current emotion exists
    augmented_emotion_dir = os.path.join(base_augmented_dir, emotion)
    if not os.path.exists(augmented_emotion_dir):
        os.makedirs(augmented_emotion_dir)
    
    # List all images in the emotion directory
    image_files = os.listdir(emotion_dir)
    
    # Iterate over each image file
    for image_file in image_files:
        # Full path to the image
        image_path = os.path.join(emotion_dir, image_file)
        
        # Read the image for augmentation
        try:
            x = io.imread(image_path)
            x = x.reshape((1, ) + x.shape)
        except:
            continue  # If an image can't be read, skip it

        # Initialize counter
        i = 0
        # Generate and save the augmented images
        for batch in datagen.flow(x, batch_size=16, save_to_dir=augmented_emotion_dir, save_prefix='aug', save_format='png'):
            i += 1
            if i > 1:  # Change this to how many augmentations you want per image
                break  # Stop after generating the desired number of augmented images


        print(f"Augmented images for {image_file} saved in {augmented_emotion_dir}")

Augmented images for fer0003147.png saved in ../dataset/augmented/surprise
Augmented images for fer0004628.png saved in ../dataset/augmented/surprise
Augmented images for fer0020946.png saved in ../dataset/augmented/surprise
Augmented images for fer0019679.png saved in ../dataset/augmented/surprise
Augmented images for fer0000328.png saved in ../dataset/augmented/surprise
Augmented images for fer0006981.png saved in ../dataset/augmented/surprise
Augmented images for fer0003621.png saved in ../dataset/augmented/surprise
Augmented images for fer0015043.png saved in ../dataset/augmented/surprise
Augmented images for fer0022823.png saved in ../dataset/augmented/surprise
Augmented images for fer0005244.png saved in ../dataset/augmented/surprise
Augmented images for fer0011231.png saved in ../dataset/augmented/surprise
Augmented images for fer0027783.png saved in ../dataset/augmented/surprise
Augmented images for fer0017126.png saved in ../dataset/augmented/surprise
Augmented images for fer0

In [35]:
num_images = len(os.listdir(f'../dataset/augmented/{emotions[0]}'))
print (num_images)

5087


In [36]:
a = num_images - 3339
print (a)

1748


In [37]:
#DELETION

import cv2
import os

# Load pre-trained Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Directory containing images
image_folder = f'../dataset/augmented/{emotions[0]}'

# Count of images deleted
deleted_count = 0

# Maximum number of images to delete
max_deleted = a

for filename in os.listdir(image_folder):
    if deleted_count >= max_deleted:
        break
    
    # Check if file is an image
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        filepath = os.path.join(image_folder, filename)
        
        # Read image
        img = cv2.imread(filepath)
        
        # Convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # If no face detected, delete the file
        if len(faces) == 0:
            os.remove(filepath)
            print(f"Deleted: {filepath}")
            deleted_count += 1

print(f"Total {deleted_count} images deleted.")


Deleted: ../dataset/augmented/surprise/aug_0_9058.png
Deleted: ../dataset/augmented/surprise/aug_0_8346.png
Deleted: ../dataset/augmented/surprise/aug_0_7075.png
Deleted: ../dataset/augmented/surprise/aug_0_7713.png
Deleted: ../dataset/augmented/surprise/aug_0_1362.png
Deleted: ../dataset/augmented/surprise/aug_0_3575.png
Deleted: ../dataset/augmented/surprise/aug_0_3561.png
Deleted: ../dataset/augmented/surprise/aug_0_7707.png
Deleted: ../dataset/augmented/surprise/aug_0_1410.png
Deleted: ../dataset/augmented/surprise/aug_0_3207.png
Deleted: ../dataset/augmented/surprise/aug_0_4568.png
Deleted: ../dataset/augmented/surprise/aug_0_2119.png
Deleted: ../dataset/augmented/surprise/aug_0_329.png
Deleted: ../dataset/augmented/surprise/aug_0_9064.png
Deleted: ../dataset/augmented/surprise/aug_0_7049.png
Deleted: ../dataset/augmented/surprise/aug_0_5886.png
Deleted: ../dataset/augmented/surprise/aug_0_4540.png
Deleted: ../dataset/augmented/surprise/aug_0_2131.png
Deleted: ../dataset/augmented

In [38]:
#MOVING FROM ORIGINAL FOLDER TO AUGMENTED FOLDER

import os
import shutil
import random

# Define source and target directories
source_dir = "../dataset/fer2013plus/train/"
target_dir = "../dataset/augmented/"

# List of emotions
emotions = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']

# Function to copy images and shuffle them
def copy_and_shuffle_images(source_dir, target_dir):
    for emotion in emotions:
        source_path = os.path.join(source_dir, emotion)
        target_path = os.path.join(target_dir, emotion)

        # Create target directory if it doesn't exist
        if not os.path.exists(target_path):
            os.makedirs(target_path)

        # Copy images from source to target
        for filename in os.listdir(source_path):
            shutil.copy(os.path.join(source_path, filename), target_path)

        # Shuffle images in target directory
        images = os.listdir(target_path)
        random.shuffle(images)
        for i, filename in enumerate(images):
            new_filename = f"{emotion}_{i}.jpg"  # You can change the format of new filenames if needed
            os.rename(os.path.join(target_path, filename), os.path.join(target_path, new_filename))

# Call the function
copy_and_shuffle_images(source_dir, target_dir)
